In [4]:
import os
import shutil
import contextlib
import pathlib
import json

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import run
import environment
import evolve
import encode
import plot

In [5]:
run1_results_dir = 'results/sector-vision_complex_sbx_100-gens/'

organism_vision_range = 1
run1_vision = environment.SectorVision(
    distance=organism_vision_range,
    distance_sectors=4,
    angle_sectors=4,
)
run1_genome_size = [run1_vision.organism_input_shape, 12, 6, 2]

fitness = evolve.EnergyFitness()

arguments = {
    'start_organism_number': 40,
    'width': 20,
    'height': 20,
    'iterations': 80 * 100,
    'generation_time': 80,
    'organism_size': 0.12,
    'food_size': 0.05,
    'organism_vision_range': organism_vision_range,
    'results_dir': run1_results_dir,
    'vision': run1_vision,
    'food_energy': 2,
    'food_appearance_number_rate': 0.7,
    'encoding': encode.RealValued(),
    'selection': evolve.TruncationSelection(fitness=fitness, n=20),
    'crossover': evolve.SBXCrossover(n=2),
    'mutation': evolve.GaussianMutation(mu=0, sigma=0.1, p=0.2),
    'elitism': 10,
    'genome_size': run1_genome_size,
    'food_particles_at_start': 40,
    'remove_dead_organisms': False,
}

# run1_params_file = os.path.join(run1_results_dir, 'data.json')

In [6]:
if os.path.exists(run1_results_dir):
    ans = input('Previous results folder exists. Do you want to delete it?')
    if ans == 'y':
        shutil.rmtree(run1_results_dir)

with contextlib.suppress(Exception):
    os.mkdir(run1_results_dir)
    
# with open(run1_params_file, 'w') as file:
#     json.dump(arguments, file)

run1 = run.OrganismsSimpleEnvironmentRunTool(**arguments)
run1.run()

Simulating organisms: 100%|██████████| 8000/8000 [23:46<00:00,  5.61it/s, Number of organisms=30, Gen=99]


In [7]:
org_loc = pq.read_table(os.path.join(run1_results_dir,
                                     'organisms_locations')).to_pandas()
org_stat = pq.read_table(os.path.join(run1_results_dir,
                                      'organisms_stats')).to_pandas()
food_loc = pq.read_table(os.path.join(run1_results_dir,
                                      'food_locations')).to_pandas()
food_stat = pq.read_table(os.path.join(run1_results_dir,
                                       'eaten_food')).to_pandas()


org_loc.reset_index(drop=True, inplace=True)
org_stat.reset_index(drop=True, inplace=True)
food_loc.reset_index(drop=True, inplace=True)
food_stat.reset_index(drop=True, inplace=True)

In [8]:
org_loc.describe()

,index,x,y,theta,iteration,generation,v,a,energy,age
count,240800.000000,240800.000000,240800.000000,240800.000000,240800.000000,240800.000000,240800.000000,240800.000000,240800.000000,240800.000000
mean,1009.436213,10.095521,10.095796,12.254387,3986.343854,49.335548,0.004671,0.002759,-4.265839,65.935216
std,579.400474,5.719967,5.836433,9.620146,2316.800348,28.958566,0.077120,0.048779,3.830180,44.333780
min,0.000000,0.000000,0.000000,-32.000000,0.000000,0.000000,-0.099778,-0.050000,-12.800000,0.000000
25%,508.000000,5.248658,4.995252,5.000000,1979.750000,24.000000,-0.083333,-0.050000,-7.200000,29.000000
50%,1010.500000,10.165341,10.337136,11.200000,3986.000000,49.000000,0.018519,0.042384,-3.600000,59.000000
75%,1512.000000,15.081456,15.060546,19.000000,5993.000000,74.000000,0.083333,0.050000,-1.200000,99.000000
max,2019.000000,20.000000,20.000000,47.800000,7999.000000,99.000000,0.099998,0.050000,9.280000,239.000000


In [9]:
org_stat.describe()

,id,age,iteration,energy
count,240800.000000,240800.000000,240800.000000,240800.000000
mean,1009.436213,65.935216,3986.343854,-4.265839
std,579.400474,44.333780,2316.800348,3.830180
min,0.000000,0.000000,0.000000,-12.800000
25%,508.000000,29.000000,1979.750000,-7.200000
50%,1010.500000,59.000000,3986.000000,-3.600000
75%,1512.000000,99.000000,5993.000000,-1.200000
max,2019.000000,239.000000,7999.000000,9.280000


In [10]:
food_stat.describe()

,organism_id,food_location,energy_taken,iteration
count,4408.00000,4408.000000,4408.0,4408.000000
mean,1096.86706,10.101629,2.0,4315.480490
std,520.69093,5.735512,0.0,2080.366114
min,0.00000,0.009828,2.0,31.000000
25%,683.00000,5.165852,2.0,2652.000000
50%,1099.00000,10.201851,2.0,4334.500000
75%,1514.00000,15.029741,2.0,5983.000000
max,2018.00000,19.991776,2.0,7978.000000


In [11]:
org_loc.groupby('generation')[['energy', 'age']].describe()

energy                                                        age   
             count      mean       std    min   25%   50%   75%   max   count   
generation                                                                      
0           3200.0 -3.188750  1.826805  -6.40 -4.72 -3.20 -1.60 -0.08  3200.0  \
1           2400.0 -5.212500  3.594527 -12.80 -8.02 -4.56 -2.24  1.04  2400.0   
2           2400.0 -5.116667  3.542092 -12.72 -7.76 -4.56 -2.16  1.20  2400.0   
3           2400.0 -4.995833  3.538286 -12.72 -7.54 -4.44 -2.08  1.92  2400.0   
4           2400.0 -4.835000  3.493765 -12.72 -7.68 -4.16 -1.84  1.84  2400.0   
...            ...       ...       ...    ...   ...   ...   ...   ...     ...   
95          2400.0 -4.289167  3.920313 -12.72 -7.68 -3.20 -1.04  3.44  2400.0   
96          2400.0 -3.833333  4.244101 -12.72 -6.96 -3.40 -0.80  8.72  2400.0   
97          2400.0 -4.188333  3.815433 -12.72 -7.12 -3.36 -1.04  3.36  2400.0   
98          2400.0 -4.605000  3.696404 -12.72 -7.36 -4.00 -1.60  3.84  2400.0   
99          2400.0 -3.880000  4.552432 -12.72 -7.86 -2.88 -0.24  5.52  2400.0   

                                                                   
                 mean        std  min    25%   50%     75%    max  
generation                                                         
0           40.500000  23.095815  1.0  20.75  40.5   60.25   80.0  
1           66.500000  44.632724  0.0  29.75  59.5  100.25  160.0  
2           66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  
3           66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  
4           66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  
...               ...        ...  ...    ...   ...     ...    ...  
95          66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  
96          66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  
97          66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  
98          66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  
99          66.166667  44.229937  0.0  29.75  59.5   99.25  159.0  

[100 rows x 16 columns]

In [12]:
# frames_dir = os.path.join(run1_results_dir, 'frames')
# frames_dir = pathlib.Path(run1_results_dir, 'frames')
# frames_dir.mkdir()
# plot.create_frames(org_loc, food_loc, arguments['width'], arguments['height'], arguments['organism_size'], arguments['food_size'], frames_dir)

In [13]:
# plot.generate_video(
#     os.path.join(run1_results_dir, 'frames/'),
#     framerate=24,
#     output=f'{run1_results_dir}evolution.mp4',
# )

# Inspecting results

In [14]:
from plotly import express as px
import plotly.graph_objs as go

In [15]:
WINDOW = 80

moving_age_mean = org_loc[[
    'iteration', 'age'
]].groupby('iteration').mean().rolling(WINDOW).mean()

moving_age_max = org_loc[['iteration', 'age'
                          ]].groupby('iteration').max().rolling(WINDOW).mean()

moving_energy_mean = org_loc[[
    'iteration', 'energy'
]].groupby('iteration').mean().rolling(WINDOW).mean()

moving_energy_max = org_loc[[
    'iteration', 'energy'
]].groupby('iteration').max().rolling(WINDOW).mean()

x = np.arange(len(moving_age_max))

ma_age = go.Scatter(
    x=x,
    y=moving_age_mean['age'],
    name=f'Moving Age Average {WINDOW}',
    mode='lines',
    legendgroup='Age',
)
mm_age = go.Scatter(x=x,
                    y=moving_age_max['age'],
                    name=f'Moving Age Max {WINDOW}',
                    mode='lines',
                    legendgroup='Age')
ma_energy = go.Scatter(x=x,
                       y=moving_energy_mean['energy'],
                       name=f'Moving Energy Average {WINDOW}',
                       mode='lines',
                       legendgroup='Energy')
mm_energy = go.Scatter(x=x,
                       y=moving_energy_max['energy'],
                       name=f'Moving Energy Max {WINDOW}',
                       mode='lines',
                       legendgroup='Energy')

layout = go.Layout(title='Evolution Statistics',
                   xaxis=dict(title='Time'))

fig = go.Figure(data=[mm_energy, ma_energy], layout=layout)
# fig = go.Figure(data=[ma_age, mm_age, mm_energy, ma_energy], layout=layout)
fig.show()

In [16]:
WINDOW = 3

moving_age_mean = org_loc[[
    'generation', 'age'
]].groupby('generation').mean().rolling(WINDOW).mean()

moving_age_max = org_loc[['generation', 'age'
                          ]].groupby('generation').max().rolling(WINDOW).mean()

moving_energy_mean = org_loc[['generation', 'energy']].groupby('generation').mean().rolling(WINDOW).mean()

moving_energy_max = org_loc[['generation', 'energy']].groupby('generation').max().rolling(WINDOW).mean()

x = np.arange(len(moving_age_max))

ma_age = go.Scatter(
    x=x,
    y=moving_age_mean['age'],
    name=f'Moving Age Average {WINDOW}',
    mode='lines',
    legendgroup='Age',
)

mm_age = go.Scatter(x=x,
                    y=moving_age_max['age'],
                    name=f'Moving Age Max {WINDOW=}',
                    mode='lines',
                    legendgroup='Age')

ma_energy = go.Scatter(x=x,
                    y=moving_energy_mean['energy'],
                    name=f'Moving Energy Max {WINDOW=}',
                    mode='lines',
                    legendgroup='Energy')

mm_energy = go.Scatter(x=x,
                    y=moving_energy_mean['energy'],
                    name=f'Moving Energy Average {WINDOW=}',
                    mode='lines',
                    legendgroup='Energy')

layout = go.Layout(title='Evolution Statistics',
                   xaxis=dict(title='Generation'))

fig = go.Figure(data=[ma_age, mm_age, mm_energy, ma_energy], layout=layout)
fig.show()

In [17]:
age_mean = org_loc[[
    'generation', 'age'
]].groupby('generation').mean()

age_max = org_loc[['generation', 'age'
                          ]].groupby('generation').max()

energy_mean = org_loc[['generation', 'energy']].groupby('generation').mean()

energy_max = org_loc[['generation', 'energy']].groupby('generation').max()

energy_max_ma = energy_max.rolling(5).mean()

x = np.arange(len(age_max))

age_avg = go.Scatter(
    x=x,
    y=age_mean['age'],
    name=f'Age Average',
    mode='lines',
    legendgroup='Age',
)

age_max = go.Scatter(x=x,
                    y=age_max['age'],
                    name=f'Age Max',
                    mode='lines',
                    legendgroup='Age')

energy_mean = go.Scatter(x=x,
                    y=energy_mean['energy'],
                    name=f'Energy Average',
                    mode='lines',
                    legendgroup='Energy')

energy_max = go.Scatter(x=x,
                    y=energy_max['energy'],
                    name=f'Energy Max',
                    mode='lines',
                    marker_color='rgba(255, 180, 0, 100)',
                    legendgroup='Energy')

energy_max_ma = go.Scatter(x=x,
                    y=energy_max_ma['energy'],
                    name=f'MA Energy Max',
                    mode='lines',
                    marker_color='rgba(255, 50, 0, 100)',
                    legendgroup='Energy')

layout = go.Layout(title='Evolution Energy Statistics',
                   xaxis=dict(title='Generation'))

fig = go.Figure(data=[energy_mean, energy_max, energy_max_ma], layout=layout)
# fig = go.Figure(data=[age_avg, age_max, energy_mean], layout=layout)

fig.show()

In [20]:
org_loc

,index,x,y,theta,name,iteration,generation,v,a,energy,age
0,1344,1.586529,14.814473,24.200000,Sed,5401,67,-0.083333,-0.05,-3.68,121
1,1346,0.725829,4.204083,24.200000,Can,5401,67,-0.083333,-0.05,-9.68,121
2,1347,1.347770,4.332456,24.200000,Mev,5401,67,-0.083333,-0.05,-7.68,121
3,1348,12.002119,0.178126,24.107526,Va,5401,67,-0.057099,-0.05,-5.68,121
4,1351,14.922490,15.904069,23.709824,He,5401,67,0.083333,0.05,-7.68,121
...,...,...,...,...,...,...,...,...,...,...,...
240795,635,5.192350,0.897127,0.000000,D,2400,30,0.000000,0.00,0.00,0
240796,636,17.032632,10.981931,0.000000,S,2400,30,0.000000,0.00,0.00,0
240797,637,9.710975,5.490141,0.000000,Hake,2400,30,0.000000,0.00,0.00,0
240798,638,1.046494,19.003067,0.000000,N,2400,30,0.000000,0.00,0.00,0


In [19]:
gen = 4

y = org_loc[org_loc.generation == gen]
# print(y[y.energy == y.energy.max()])
best_id = y[y.energy == y.energy.max()]
print(best_id, type(best_id))
best = y[y['index'] == best_id['index']]
# best = best.sort_values(by='iteration')
# worst_id = y[y.energy == y.energy.min()]['index']
# worst = y[y['index'] == worst_id]
# worst = worst.sort_values(by='iteration')

        index         x         y  theta name  iteration  generation   
202442    102  2.490961  7.994988   -0.4    L        322           4  \

               v     a  energy  age  
202442  0.006944  0.05    1.84    2   <class 'pandas.core.frame.DataFrame'>


ValueError: Can only compare identically-labeled Series objects

In [ ]:
# age_mean = org_loc[[
#     'generation', 'age'
# ]].groupby('generation').mean()
# age_max = org_loc[['generation', 'age'
#                           ]].groupby('generation').max()
# energy_mean = org_loc[['generation', 'energy']].groupby('generation').mean()

x = np.arange(len(org_stat))
gens = org_loc['generation']

fig = go.Figure()

for gen in gens:
  y = org_loc[org_loc.generation == gen]
  best_id = y[y.energy == y.energy.max()].index
  best = y[y.index == best_id]
  best = best.sort_values(by='iteration')
  worst_id = y[y.energy == y.energy.min()].index
  worst = y[y.index == worst_id]
  worst = worst.sort_values(by='iteration')
  # y = org_stat[org_loc['id'] == id].sort_values(by='iteration')
  fig.add_trace(go.Scatter(x=x, y=best['energy']))
  fig.add_trace(go.Scatter(x=x, y=worst['energy']))

# age_avg = go.Scatter(
#     x=x,
#     y=age_mean['age'],
#     name=f'Age Average',
#     mode='lines',
#     legendgroup='Age',
# )

# age_max = go.Scatter(x=x,
#                     y=age_max['age'],
#                     name=f'Age Max',
#                     mode='lines',
#                     legendgroup='Age')

# energy_mean = go.Scatter(x=x,
#                     y=energy_mean['energy'],
#                     name=f'Energy Average',
#                     mode='lines',
#                     legendgroup='Energy')

# layout = go.Layout(title='Evolution Statistics',
#                    xaxis=dict(title='Generation'))

# fig = go.Figure(data=[energy_mean], layout=layout)
# fig = go.Figure(data=[age_avg, age_max, energy_mean], layout=layout)

fig.show()

ValueError: Lengths must match to compare